# QUESTION 1

#### Firstable we import the data to our notebook 

In [67]:



#firstable we import the data to our notebook 
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0



df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Droping Borough not assigned

In [3]:
#droping Borough not assigned
import numpy as np
df_data_1.replace("Not assigned",np.nan,inplace= True)
df_data_1.dropna(subset=["Borough"], axis=0, inplace=True)
df_data_1.reset_index(drop=True, inplace=True)
df_data_1.head()


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### Replace Neighbourhood not assigned with the value of the Borough

In [4]:
#replace Neighbourhood not assigned [with the value of the Borough
df_data_1["Neighbourhood"].replace(np.nan, df_data_1["Borough"], inplace=True)
df_data_1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### Combine neighbourhood with the same Post Code

In [5]:
#Combine neighbourhood with the same Post Code
df_group = df_data_1[['Postcode','Borough','Neighbourhood']]
df_group = df_group.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)
df_group = df_group.sample(frac=1).reset_index()
df_group['Neighbourhood']=df_group['Neighbourhood'].str.join(',')
df_group.head()

,Postcode,Borough,Neighbourhood
0,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town"
2,M9W,Etobicoke,Northwest
3,M2L,North York,"Silver Hills,York Mills"
4,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto"


#### Shape of Dataframe

In [6]:
#shape of the dataframe
df_group.shape

(103, 3)

# Question 2

#### We get in the notebook the geospatial coordinates

In [7]:

# we get in the notebook the geospatial coordinates
body = client_6e7bb81ad7a14c728e0d0337880e2b32.get_object(Bucket='segmentingandclusteringintoronto-donotdelete-pr-fvolvfnvxbsxwn',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.rename(columns={'"Postal Code"':'Postcode'}, inplace=True)
df_data_2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Full dataframe with coordinates

In [8]:
#Full dataframe with coordinates
df_Toronto=df_group.merge(df_data_2, left_on='Postcode',right_on='Postal Code')
df_Total=df_Toronto[['Postcode','Borough','Neighbourhood','Latitude','Longitude']]
df_Total.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
2,M9W,Etobicoke,Northwest,43.706748,-79.594054
3,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714
4,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321


In [9]:
df_Total.shape

(103, 5)

# Question 3

#### Importing Libraries

In [12]:
## QUESTION 3

#importing libraries
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         673 KB

The following NEW packages will be INSTALLED:

    altair:  4.0.1-py_0 conda-forge
    branca:  0.3.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


foli

#### Toronto Geolocation

In [13]:
#Toronto geolocation

address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create map of Toronto using latitude and longitude values

In [20]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(df_Total['Latitude'], df_Total['Longitude'], df_Total['Borough'], df_Total['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Looking for Borough Downtown Toronto in order to cluster that


In [24]:
#looking for Borough Downtown Toronto in order to cluster that

downtown_data = df_Total[df_Total['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
1,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
2,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5X,Downtown Toronto,"First Canadian Place,Underground city",43.648429,-79.382280


#### Let's get the geographical coordinates of Downtown Toronto

In [25]:
#Let's get the geographical coordinates of Downtown Toronto
address = 'Downtown Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


#### Create map of Downtown Toronto using latitude and longitude values

In [26]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

#### Define Foursquare Credentials and Version

In [27]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'EU0UYEMIJL41HG5HF1PJ4DLNPVRPH0RFK1WEL1A1E1B2D5QE' # your Foursquare ID
CLIENT_SECRET = 'OA4PW4R4KDNPUWRDU3T4PNVW5EIIKUZAPN1CUZX3FUG41BLA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EU0UYEMIJL41HG5HF1PJ4DLNPVRPH0RFK1WEL1A1E1B2D5QE
CLIENT_SECRET:OA4PW4R4KDNPUWRDU3T4PNVW5EIIKUZAPN1CUZX3FUG41BLA


#### Let's explore the first neighborhood in our dataframe.

In [28]:
#Let's explore the first neighborhood in our dataframe.
downtown_data.loc[0, 'Neighbourhood']

'Cabbagetown,St. James Town'

#### Get the neighborhood's latitude and longitude values.

In [30]:
#Get the neighborhood's latitude and longitude values.
neighbourhood_latitude = downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = downtown_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = downtown_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Cabbagetown,St. James Town are 43.667967, -79.3676753.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [31]:
#Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=EU0UYEMIJL41HG5HF1PJ4DLNPVRPH0RFK1WEL1A1E1B2D5QE&client_secret=OA4PW4R4KDNPUWRDU3T4PNVW5EIIKUZAPN1CUZX3FUG41BLA&v=20180605&ll=43.667967,-79.3676753&radius=500&limit=100'

#### Send the GET request and examine the results

In [32]:
#Send the GET request and examine the resutls

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e371bb91835dd001b1178bb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Cabbagetown',
  'headerFullLocation': 'Cabbagetown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 43,
  'suggestedBounds': {'ne': {'lat': 43.6724670045, 'lng': -79.3614658826597},
   'sw': {'lat': 43.663466995499995, 'lng': -79.3738847173403}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b646a6ff964a5205cb12ae3',
       'name': 'Cranberries',
       'location': {'address': '601 Parliament St.',
        'crossStreet': 'at Wellesley St. E',
        'lat': 43.6678427705951,
        'lng': -79.36940687874281,
        'labeledLatLngs': [{'lab

#### Function that extracts the category of the venue

In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now we are ready to clean the json and structure it into a pandas dataframe.

In [35]:
#Now we are ready to clean the json and structure it into a pandas dataframe.

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Cranberries,Diner,43.667843,-79.369407
1,F'Amelia,Italian Restaurant,43.667536,-79.368613
2,Kingyo Toronto,Japanese Restaurant,43.665895,-79.368415
3,Butter Chicken Factory,Indian Restaurant,43.667072,-79.369184
4,Murgatroid,Restaurant,43.667381,-79.369311


#### And how many venues were returned by Foursquare

In [36]:
#And how many venues were returned by Foursquare

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

43 venues were returned by Foursquare.


#### Explore Neighborhoods in Downtown Toronto

In [37]:
#Explore Neighborhoods in Downtown Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called downtown_venues.

In [40]:
#Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.

downtown_venues = getNearbyVenues(names=downtown_data['Neighbourhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )



Cabbagetown,St. James Town
Adelaide,King,Richmond
Harbord,University of Toronto
Berczy Park
First Canadian Place,Underground city
Design Exchange,Toronto Dominion Centre
Stn A PO Boxes 25 The Esplanade
Christie
Church and Wellesley
Commerce Court,Victoria Hotel
Queen's Park
Chinatown,Grange Park,Kensington Market
Ryerson,Garden District
St. James Town
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Central Bay Street
Rosedale
Harbourfront
Harbourfront East,Toronto Islands,Union Station


#### Let's check the size of the resulting dataframe

In [41]:
#Let's check the size of the resulting dataframe

print(downtown_venues.shape)
downtown_venues.head()

(1300, 7)


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Cabbagetown,St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
1,"Cabbagetown,St. James Town",43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
2,"Cabbagetown,St. James Town",43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
3,"Cabbagetown,St. James Town",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
4,"Cabbagetown,St. James Town",43.667967,-79.367675,Murgatroid,43.667381,-79.369311,Restaurant


#### Let's check how many venues were returned for each neighborhood

In [43]:
#Let's check how many venues were returned for each neighborhood

downtown_venues.groupby('Neighborhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",43,43,43,43,43,43
Central Bay Street,82,82,82,82,82,82
"Chinatown,Grange Park,Kensington Market",86,86,86,86,86,86
Christie,17,17,17,17,17,17
Church and Wellesley,83,83,83,83,83,83
"Commerce Court,Victoria Hotel",100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [44]:
#Let's find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 197 uniques categories.


#### Analyze Each Neighborhood

In [46]:
#Analyze Each Neighborhood

# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
manhattan_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Cabbagetown,St. James Town"
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Cabbagetown,St. James Town"
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Cabbagetown,St. James Town"
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Cabbagetown,St. James Town"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Cabbagetown,St. James Town"


#### And let's examine the new dataframe size.

In [47]:
#And let's examine the new dataframe size.

downtown_onehot.shape

(1300, 198)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [48]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.01,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.00,0.00,0.017857,0.000000,0.000000,0.000000,0.00,0.000000
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.071429,0.071429,0.142857,0.214286,0.142857,0.000000,0.000000,0.00,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,0.000000,0.00,...,0.000000,0.00,0.00,0.00,0.012195,0.000000,0.000000,0.012195,0.00,0.012195
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.00,0.00,0.046512,0.000000,0.069767,0.011628,0.00,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
7,Church and Wellesley,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,0.000000,0.00,...,0.012048,0.00,0.00,0.00,0.000000,0.000000,0.012048,0.000000,0.00,0.012048
8,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.00,0.000000
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,...,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.00,0.000000


#### Let's print each neighborhood along with the top 5 most common venues

In [50]:
#Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in downtown_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0       Coffee Shop  0.07
1        Steakhouse  0.04
2              Café  0.04
3        Restaurant  0.03
4  Asian Restaurant  0.03


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2          Bakery  0.04
3  Farmers Market  0.04
4        Beer Bar  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0     Airport Service  0.21
1      Airport Lounge  0.14
2    Airport Terminal  0.14
3             Airport  0.07
4  Airport Food Court  0.07


----Cabbagetown,St. James Town----
                venue  freq
0         Coffee Shop  0.07
1         Pizza Place  0.07
2  Italian Restaurant  0.05
3                Café  0.05
4                 Pub  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1  Italian Restaurant  0.05
2        Burger Joint  0.04
3                Café 

#### Let's put that into a pandas dataframe

In [51]:
#Let's put that into a pandas dataframe

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [53]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Steakhouse,Café,Restaurant,Bar,Bakery,Cosmetics Shop,Thai Restaurant,Asian Restaurant,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Bakery,Cheese Shop,Steakhouse,Seafood Restaurant,Beer Bar,Farmers Market,Concert Hall
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Bar,Sculpture Garden,Airport,Airport Food Court
3,"Cabbagetown,St. James Town",Coffee Shop,Pizza Place,Restaurant,Pub,Café,Italian Restaurant,Bakery,Gastropub,Butcher,Plaza
4,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Sandwich Place,Burger Joint,Café,Ice Cream Shop,Department Store,Salad Place


#### Cluster Neighborhoods

In [54]:
#Cluster Neighborhoods

# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 3, 0, 4, 0, 0, 0], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [55]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtown_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,0,Coffee Shop,Pizza Place,Restaurant,Pub,Café,Italian Restaurant,Bakery,Gastropub,Butcher,Plaza
1,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568,0,Coffee Shop,Steakhouse,Café,Restaurant,Bar,Bakery,Cosmetics Shop,Thai Restaurant,Asian Restaurant,Burger Joint
2,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049,0,Café,Bar,Japanese Restaurant,Restaurant,Bookstore,Bakery,Nightclub,Noodle House,Coffee Shop,Chinese Restaurant
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Café,Bakery,Cheese Shop,Steakhouse,Seafood Restaurant,Beer Bar,Farmers Market,Concert Hall
4,M5X,Downtown Toronto,"First Canadian Place,Underground city",43.648429,-79.382280,0,Coffee Shop,Café,Restaurant,Steakhouse,Asian Restaurant,Gastropub,Seafood Restaurant,Gym,Hotel,Deli / Bodega


#### Finally, let's visualize the resulting clusters

In [56]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [58]:
#Examine Clusters

#Cluster 1

downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pizza Place,Restaurant,Pub,Café,Italian Restaurant,Bakery,Gastropub,Butcher,Plaza
1,Downtown Toronto,0,Coffee Shop,Steakhouse,Café,Restaurant,Bar,Bakery,Cosmetics Shop,Thai Restaurant,Asian Restaurant,Burger Joint
2,Downtown Toronto,0,Café,Bar,Japanese Restaurant,Restaurant,Bookstore,Bakery,Nightclub,Noodle House,Coffee Shop,Chinese Restaurant
3,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Café,Bakery,Cheese Shop,Steakhouse,Seafood Restaurant,Beer Bar,Farmers Market,Concert Hall
4,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Steakhouse,Asian Restaurant,Gastropub,Seafood Restaurant,Gym,Hotel,Deli / Bodega
5,Downtown Toronto,0,Coffee Shop,Café,Hotel,Steakhouse,Restaurant,Italian Restaurant,Bar,Gastropub,Seafood Restaurant,Deli / Bodega
6,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Cocktail Bar,Japanese Restaurant,Italian Restaurant,Hotel,Beer Bar,Seafood Restaurant,Farmers Market
8,Downtown Toronto,0,Coffee Shop,Gay Bar,Sushi Restaurant,Japanese Restaurant,Restaurant,Fast Food Restaurant,Café,Grocery Store,Gym,Hotel
9,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Gym,Bakery,Seafood Restaurant,Steakhouse,Italian Restaurant,Deli / Bodega
11,Downtown Toronto,0,Vietnamese Restaurant,Café,Chinese Restaurant,Bar,Dumpling Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Burger Joint,Donut Shop


#### Cluster 2

In [60]:
#Cluster 2

downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Downtown Toronto,1,Park,Playground,Trail,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


#### Cluster 3

In [61]:
#cluster 3

downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Bar,Sculpture Garden,Airport,Airport Food Court


#### Cluster 4

In [62]:
#cluster 4

downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,3,Coffee Shop,Park,Gym,Burger Joint,Salad Place,Portuguese Restaurant,Sandwich Place,Juice Bar,Chinese Restaurant,Burrito Place
15,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Sandwich Place,Burger Joint,Café,Ice Cream Shop,Department Store,Salad Place
17,Downtown Toronto,3,Coffee Shop,Pub,Bakery,Park,Café,Restaurant,Mexican Restaurant,Farmers Market,Performing Arts Venue,Ice Cream Shop
18,Downtown Toronto,3,Coffee Shop,Aquarium,Italian Restaurant,Hotel,Café,Fried Chicken Joint,Restaurant,Brewery,Scenic Lookout,Pizza Place


#### Cluster 5

In [63]:
#cluster 5

downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,4,Grocery Store,Café,Park,Nightclub,Coffee Shop,Restaurant,Gas Station,Baby Store,Italian Restaurant,Diner
